In [4]:
import numpy as np
import pygad
import tree_representation
import csv

In [9]:
with open("dataset.csv") as file:
    reader = csv.reader(file)

    for row in reader:
        xs = np.fromstring(row[1][1:-1], sep=", ")
        ys = np.fromstring(row[2][1:-1], sep=", ")


def fitness_func(ga_instance, solution, solution_idx):
  return tree_representation.fitness(solution, xs, ys)
  

class CustomIndividual:
    def __init__(self, gene):
        self.gene = gene
        self.fitness = self.calculate_fitness()

    def calculate_fitness(self):
        # Implement your custom fitness calculation logic here.
        return 0.0 


initial_population = [CustomIndividual(1) for _ in range(50)]


def crossover_func(parents, offspring_size, ga_instance):
  assert offspring_size == 2
  
  offspring = []

  t1, t2 = tree_representation.crossover_tree(parents[0], parents[1])

  # while len(offspring) != offspring_size[0]:
  #    split_point = np.random.randint(0, 1)
  #    offspring.append(np.array([parents[0, split_point], parents[1, 1-split_point]]))
  
  # offspring.append(np.array([parents[0][0], parents[1][1]]))
  # offspring.append(np.array([parents[1][0], parents[0][1]]))

  offspring.append(t1)
  offspring.append(t2)

  return offspring


def mutation_func(offspring, ga_instance):
    print("OFFSPRING", offspring)
    
    # for chromosome_idx in range(offspring.shape[0]):
    #    offspring[chromosome_idx, 0] += np.random.randint(-5, 5)
    #    offspring[chromosome_idx, 1] += np.random.randint(-5, 5)
    return offspring


ga_instance = pygad.GA(num_generations=100,
                       num_parents_mating=2,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       crossover_type=crossover_func,
                       mutation_type=mutation_func,
                       )

ga_instance.run()

solution, solution_fitness, solution_idx = ga_instance.best_solution()

solution, solution_fitness

A 2D list is expected to the initail_population parameter but a (1-D) list found.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/pygad/pygad.py", line 448, in __init__
    raise ValueError(f"A 2D list is expected to the initail_population parameter but a ({numpy.array(initial_population).ndim}-D) list found.")
ValueError: A 2D list is expected to the initail_population parameter but a (1-D) list found.


AttributeError: 'tuple' object has no attribute 'tb_frame'